In [1]:
from keras.models import *
from keras.layers import *
import keras
import os
import numpy as np

from matplotlib import pyplot as plt
from IPython.display import clear_output

/Users/jaejunlee/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
test_data = np.loadtxt(os.getcwd()+'/test.wav.txt')
test_data.size

4000

In [48]:
model_name = 'RES8_NARROW';

config = dict(input_shape=(100,40,1,), conv_size=(3,3), conv_stride=(1,1), n_labels=12, n_layers=6, n_feature_maps=19, res_pool=(4, 3), use_dilation=False)
# 	RES8_NARROW : {
# 		input_shape : [40, 100, 1],
# 		n_labels : 12,
# 		n_layers : 6,
# 		n_feature_maps : 19,
# 		res_pool : [3, 4],
# 		use_dilation : false,
# 		conv_size : [3, 3],
# 		conv_stride : [1, 1]
# 	},
config

{'conv_size': (3, 3),
 'conv_stride': (1, 1),
 'input_shape': (100, 40, 1),
 'n_feature_maps': 19,
 'n_labels': 12,
 'n_layers': 6,
 'res_pool': (4, 3),
 'use_dilation': False}

In [4]:
layers = {}

In [5]:
# conv0 = tf.layers.conv2d({
#     filters: this.config['n_feature_maps'],
#     kernelSize: this.config['conv_size'],
#     strides: this.config['conv_stride'],
#     padding: "same",
#     useBias: false,
#     kernelInitializer: tf.initializers.zeros(),
#     biasInitializer: tf.initializers.zeros(),
# })
layers['conv0'] = Conv2D(
    filters=config['n_feature_maps'],
    kernel_size=config['conv_size'],
    strides=config['conv_stride'],
    padding='same',
    use_bias=False,
    activation='relu',
    kernel_initializer='glorot_uniform',
    bias_initializer=keras.initializers.zeros(),
    name = 'conv0'
)

layers

{'conv0': <keras.layers.convolutional.Conv2D at 0xb28e82d50>}

In [6]:
# this.pool = tf.layers.averagePooling2d({
#     poolSize: this.config['res_pool']
# })
layers['pool'] = AveragePooling2D(
    pool_size=config['res_pool'], 
    strides=None,
    data_format=None,
    name = 'pool')

layers

{'conv0': <keras.layers.convolutional.Conv2D at 0xb28e82d50>,
 'pool': <keras.layers.pooling.AveragePooling2D at 0xb28e82fd0>}

In [7]:
# for (var i  = 0; i < (this.config['n_layers']); i++) {
#     this['conv'+ (i+1)] = tf.layers.conv2d({
#         filters: this.config['n_feature_maps'],
#         // inputShape: this.config['n_feature_maps'],
#         kernelSize: this.config['conv_size'],
#         padding: "same",
#         dilation: 1,
#         useBias: false,
#         kernelInitializer: tf.initializers.zeros(),
#         biasInitializer: tf.initializers.zeros(),
#     })
# }

# ("conv{}".format(i + 1), conv

for i in range(config['n_layers']):
    conv = Conv2D(
        filters=config['n_feature_maps'],
        kernel_size=config['conv_size'],
        strides=config['conv_stride'],
        padding='same',
        use_bias=False,
        activation='relu',
        kernel_initializer='glorot_uniform',
        bias_initializer=keras.initializers.zeros(),
        name = "conv{}".format(i + 1)
    )
    layers.update({"conv{}".format(i + 1) : conv})

layers

{'conv0': <keras.layers.convolutional.Conv2D at 0xb28e82d50>,
 'conv1': <keras.layers.convolutional.Conv2D at 0xb28ea1750>,
 'conv2': <keras.layers.convolutional.Conv2D at 0xb28ea1810>,
 'conv3': <keras.layers.convolutional.Conv2D at 0xb28ea1910>,
 'conv4': <keras.layers.convolutional.Conv2D at 0xb28ea1a10>,
 'conv5': <keras.layers.convolutional.Conv2D at 0xb28ea1b10>,
 'conv6': <keras.layers.convolutional.Conv2D at 0xb28ea1c10>,
 'pool': <keras.layers.pooling.AveragePooling2D at 0xb28e82fd0>}

In [8]:
# for (var i  = 0; i < (this.config['n_layers']); i++) {
#     this['bn'+ (i+1)] = tf.layers.batchNormalization({
#         epsilon: 0.00001,
#         momentum: 0.1,
#         gammaInitializer: tf.initializers.ones(),
#         betaInitializer: tf.initializers.zeros(),
#     })
# }

for i in range(config['n_layers']):
    bn = BatchNormalization(
        momentum=0.1, 
        epsilon=0.00001, 
        gamma_initializer=keras.initializers.ones(), 
        beta_initializer=keras.initializers.zeros(),
        name="bn{}".format(i)
    )
    layers.update({"bn{}".format(i+1) : bn})

layers

{'bn1': <keras.layers.normalization.BatchNormalization at 0xb28ea1410>,
 'bn2': <keras.layers.normalization.BatchNormalization at 0xb28ea1510>,
 'bn3': <keras.layers.normalization.BatchNormalization at 0xb28ea1690>,
 'bn4': <keras.layers.normalization.BatchNormalization at 0xb28ea1dd0>,
 'bn5': <keras.layers.normalization.BatchNormalization at 0xb28ea1f50>,
 'bn6': <keras.layers.normalization.BatchNormalization at 0xb28ea5090>,
 'conv0': <keras.layers.convolutional.Conv2D at 0xb28e82d50>,
 'conv1': <keras.layers.convolutional.Conv2D at 0xb28ea1750>,
 'conv2': <keras.layers.convolutional.Conv2D at 0xb28ea1810>,
 'conv3': <keras.layers.convolutional.Conv2D at 0xb28ea1910>,
 'conv4': <keras.layers.convolutional.Conv2D at 0xb28ea1a10>,
 'conv5': <keras.layers.convolutional.Conv2D at 0xb28ea1b10>,
 'conv6': <keras.layers.convolutional.Conv2D at 0xb28ea1c10>,
 'pool': <keras.layers.pooling.AveragePooling2D at 0xb28e82fd0>}

In [9]:
# this.output = tf.layers.dense({
#     units: this.config['n_labels'],
#     activation: 'linear',
#     biasInitializer : tf.initializers.zeros()
# });

layers['output'] = Dense(
    config['n_labels'],
    activation='linear', #None
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    name="output"
)

layers

{'bn1': <keras.layers.normalization.BatchNormalization at 0xb28ea1410>,
 'bn2': <keras.layers.normalization.BatchNormalization at 0xb28ea1510>,
 'bn3': <keras.layers.normalization.BatchNormalization at 0xb28ea1690>,
 'bn4': <keras.layers.normalization.BatchNormalization at 0xb28ea1dd0>,
 'bn5': <keras.layers.normalization.BatchNormalization at 0xb28ea1f50>,
 'bn6': <keras.layers.normalization.BatchNormalization at 0xb28ea5090>,
 'conv0': <keras.layers.convolutional.Conv2D at 0xb28e82d50>,
 'conv1': <keras.layers.convolutional.Conv2D at 0xb28ea1750>,
 'conv2': <keras.layers.convolutional.Conv2D at 0xb28ea1810>,
 'conv3': <keras.layers.convolutional.Conv2D at 0xb28ea1910>,
 'conv4': <keras.layers.convolutional.Conv2D at 0xb28ea1a10>,
 'conv5': <keras.layers.convolutional.Conv2D at 0xb28ea1b10>,
 'conv6': <keras.layers.convolutional.Conv2D at 0xb28ea1c10>,
 'output': <keras.layers.core.Dense at 0xb28e82850>,
 'pool': <keras.layers.pooling.AveragePooling2D at 0xb28e82fd0>}

In [10]:
# this.add = tf.layers.add();
layers['add'] = Add();

layers

{'add': <keras.layers.merge.Add at 0xb28e82650>,
 'bn1': <keras.layers.normalization.BatchNormalization at 0xb28ea1410>,
 'bn2': <keras.layers.normalization.BatchNormalization at 0xb28ea1510>,
 'bn3': <keras.layers.normalization.BatchNormalization at 0xb28ea1690>,
 'bn4': <keras.layers.normalization.BatchNormalization at 0xb28ea1dd0>,
 'bn5': <keras.layers.normalization.BatchNormalization at 0xb28ea1f50>,
 'bn6': <keras.layers.normalization.BatchNormalization at 0xb28ea5090>,
 'conv0': <keras.layers.convolutional.Conv2D at 0xb28e82d50>,
 'conv1': <keras.layers.convolutional.Conv2D at 0xb28ea1750>,
 'conv2': <keras.layers.convolutional.Conv2D at 0xb28ea1810>,
 'conv3': <keras.layers.convolutional.Conv2D at 0xb28ea1910>,
 'conv4': <keras.layers.convolutional.Conv2D at 0xb28ea1a10>,
 'conv5': <keras.layers.convolutional.Conv2D at 0xb28ea1b10>,
 'conv6': <keras.layers.convolutional.Conv2D at 0xb28ea1c10>,
 'output': <keras.layers.core.Dense at 0xb28e82850>,
 'pool': <keras.layers.pooling.A

In [11]:
# this.globalAvgPool = tf.layers.globalAveragePooling2d({});
layers['globalAvgPool'] = GlobalAveragePooling2D();

layers

{'add': <keras.layers.merge.Add at 0xb28e82650>,
 'bn1': <keras.layers.normalization.BatchNormalization at 0xb28ea1410>,
 'bn2': <keras.layers.normalization.BatchNormalization at 0xb28ea1510>,
 'bn3': <keras.layers.normalization.BatchNormalization at 0xb28ea1690>,
 'bn4': <keras.layers.normalization.BatchNormalization at 0xb28ea1dd0>,
 'bn5': <keras.layers.normalization.BatchNormalization at 0xb28ea1f50>,
 'bn6': <keras.layers.normalization.BatchNormalization at 0xb28ea5090>,
 'conv0': <keras.layers.convolutional.Conv2D at 0xb28e82d50>,
 'conv1': <keras.layers.convolutional.Conv2D at 0xb28ea1750>,
 'conv2': <keras.layers.convolutional.Conv2D at 0xb28ea1810>,
 'conv3': <keras.layers.convolutional.Conv2D at 0xb28ea1910>,
 'conv4': <keras.layers.convolutional.Conv2D at 0xb28ea1a10>,
 'conv5': <keras.layers.convolutional.Conv2D at 0xb28ea1b10>,
 'conv6': <keras.layers.convolutional.Conv2D at 0xb28ea1c10>,
 'globalAvgPool': <keras.layers.pooling.GlobalAveragePooling2D at 0xb28e82450>,
 'out

In [12]:
# tf.input({shape: this.config['input_shape']});
input = Input(shape=config['input_shape'], name='input')
x = input;

# x = x.unsqueeze(1)
for i in range(config['n_layers'] + 1):
#     y = F.relu(getattr(self, "conv{}".format(i))(x))
    y = layers["conv{}".format(i)](x)
    if i == 0:
#         if hasattr(self, "pool"):
        if 'pool' in layers:
#             y = self.pool(y)
            y = layers['pool'](y)
        old_x = y
        
    if i > 0 and i % 2 == 0:
#         x = y + old_x
        x = layers['add']([y, old_x])
        old_x = x
    else:
        x = y
        
    if i > 0:
#         x = getattr(self, "bn{}".format(i))(x)
        x = layers["bn{}".format(i)](x)
    
# x = x.view(x.size(0), x.size(1), -1) # shape: (batch, feats, o3)
# x = torch.mean(x, 2)
x = layers['globalAvgPool'](x)

# return self.output(x)
output = layers['output'](x)

model = Model(inputs=input, outputs=output)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 100, 40, 1)   0                                            
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 100, 40, 19)  171         input[0][0]                      
__________________________________________________________________________________________________
pool (AveragePooling2D)         (None, 25, 13, 19)   0           conv0[0][0]                      
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 25, 13, 19)   3249        pool[0][0]                       
__________________________________________________________________________________________________
bn0 (Batch

In [13]:
for layer in model.layers:
    print layer.name
    print "\t Input shape : " + str(layer.input_shape)
    print "\t Output shape: "+str(layer.output_shape)

input
	 Input shape : (None, 100, 40, 1)
	 Output shape: (None, 100, 40, 1)
conv0
	 Input shape : (None, 100, 40, 1)
	 Output shape: (None, 100, 40, 19)
pool
	 Input shape : (None, 100, 40, 19)
	 Output shape: (None, 25, 13, 19)
conv1
	 Input shape : (None, 25, 13, 19)
	 Output shape: (None, 25, 13, 19)
bn0
	 Input shape : (None, 25, 13, 19)
	 Output shape: (None, 25, 13, 19)
conv2
	 Input shape : (None, 25, 13, 19)
	 Output shape: (None, 25, 13, 19)
add_1
	 Input shape : [(None, 25, 13, 19), (None, 25, 13, 19)]
	 Output shape: (None, 25, 13, 19)
bn1
	 Input shape : (None, 25, 13, 19)
	 Output shape: (None, 25, 13, 19)
conv3
	 Input shape : (None, 25, 13, 19)
	 Output shape: (None, 25, 13, 19)
bn2
	 Input shape : (None, 25, 13, 19)
	 Output shape: (None, 25, 13, 19)
conv4
	 Input shape : (None, 25, 13, 19)
	 Output shape: (None, 25, 13, 19)
bn3
	 Input shape : (None, 25, 13, 19)
	 Output shape: (None, 25, 13, 19)
conv5
	 Input shape : (None, 25, 13, 19)
	 Output shape: (None, 25, 13, 1

In [24]:
#  const optimizer = tf.train.momentum({
#     learningRate: 0.1,
#     momentum: 0.9,
#     useNesterov: true
# });
# this.model.compile({
#     optimizer: optimizer,
#     loss: 'categoricalCrossentropy',
#     metrics: ['accuracy'],
# });

# optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True)
optimizer = keras.optimizers.SGD(
    lr=0.0001, 
    decay=1e-5, 
    momentum=0.9, 
    nesterov=True
)
model.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizer, 
    metrics=['accuracy']
)

In [25]:
sample_batch_size = 10;

sample_data = test_data.reshape(config['input_shape'])

sample_label = np.zeros(config['n_labels']);
sample_label[0] = 1

batch_x = np.stack([sample_data] * sample_batch_size, axis = 0);
batch_y = np.tile(sample_label,(sample_batch_size, 1))

assert not np.any(np.isnan(batch_x))
assert not np.any(np.isnan(batch_y))

print batch_x.shape
print batch_y.shape

(10, 100, 40, 1)
(10, 12)


In [26]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

In [27]:
model.fit(
    batch_x, 
    batch_y, 
    epochs=10, 
    batch_size=sample_batch_size
)

# model.fit(
#     batch_x, 
#     batch_y, 
#     epochs=10, 
#     batch_size=sample_batch_size, 
#     callbacks=[plot_losses],
#     verbose=0)

Epoch 1/10
10/10 [==============================] - 1s 69ms/step - loss: 16.1181 - acc: 1.0000
Epoch 2/10
10/10 [==============================] - 0s 6ms/step - loss: 16.1181 - acc: 1.0000
Epoch 3/10
10/10 [==============================] - 0s 6ms/step - loss: 16.1181 - acc: 1.0000
Epoch 4/10
10/10 [==============================] - 0s 6ms/step - loss: 16.1181 - acc: 1.0000
Epoch 5/10
10/10 [==============================] - 0s 6ms/step - loss: 16.1181 - acc: 1.0000
Epoch 6/10
10/10 [==============================] - 0s 7ms/step - loss: 16.1181 - acc: 1.0000
Epoch 7/10
10/10 [==============================] - 0s 6ms/step - loss: 16.1181 - acc: 1.0000
Epoch 8/10
10/10 [==============================] - 0s 6ms/step - loss: 16.1181 - acc: 1.0000
Epoch 9/10
10/10 [==============================] - 0s 6ms/step - loss: 16.1181 - acc: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 6ms/step - loss: 16.1181 - acc: 1.0000


In [84]:
output = model.predict(np.array([sample_data]))
print 'output : ', output
prediction = np.argmax(output)
print 'prediction : ', prediction

output :  [[-1.7045540e-04  2.3145785e-04  4.5049863e-05  1.2632887e-04
   5.3954849e-05  3.5216400e-04  1.3520155e-04 -3.2388768e-04
  -1.4886493e-04 -3.2141441e-05  1.2686354e-05  1.7677298e-04]]
prediction :  5


In [67]:
for key, value in layers.iteritems():
    weights = np.array(value.get_weights())
    for i in range(len(weights)):
        if (weights.shape[0] != 0):
            weights[i].tofile('../weights/'+model_name+'/'+key+'_'+str(i)+'.txt', ", ")

In [68]:
content = 'const weights = {\n\t'+model_name+' : {\n';

for key, value in layers.iteritems():
    weights = np.array(value.get_weights())
    
    for i in range(len(weights)):
        if (weights.shape[0] != 0):
            file = open('../weights/'+model_name+'/'+key+'_'+str(i)+'.txt', 'r')
            content += ('\t\t ' + key+'_'+str(i) + ' : [' + file.read() + '], \n')

content += ('\t}\n}\n')

file = open('../weights/'+model_name+'/'+ 'weights.js','w')
file.write(content) 
file.close()